# Initiliation

In [ ]:
!pip install transformers datasets --quiet

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/Baseline'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/Baseline


In [ ]:
dfPJ = pd.read_csv('myPers_OPN_Baseline.csv', sep=",", error_bad_lines=False)
dfPJ

,text,label
0,likes the sound of thunder.,1
1,is so sleepy it's not even funny that's she ca...,1
2,is sore and wants the knot of muscles at the b...,1
3,likes how the day sounds in this new song.,1
4,is home. <3,1
...,...,...
9912,little things give you away.,1
9913,is wishing it was Saturday.,1
9914,is studying hard for the G.R.E.,1
9915,snipers get more head,1


In [ ]:
df_clean= []
for i,row in dfPJ.iterrows():
  df_clean.append({
      'text': str(row['text']),
      'label': int(row['label']),
  })

dfPJ = pd.DataFrame(df_clean)
dfPJ

,text,label
0,likes the sound of thunder.,1
1,is so sleepy it's not even funny that's she ca...,1
2,is sore and wants the knot of muscles at the b...,1
3,likes how the day sounds in this new song.,1
4,is home. <3,1
...,...,...
9912,little things give you away.,1
9913,is wishing it was Saturday.,1
9914,is studying hard for the G.R.E.,1
9915,snipers get more head,1


# Model Training

In [ ]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [ ]:
train, test = train_test_split(dfPJ, test_size=0.2, random_state=0, stratify=dfPJ.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_OPN", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [ ]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 13.3 MB/s 
     |████████████████████████████████| 80 kB 8.1 MB/s 
     |████████████████████████████████| 209 kB 54.8 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 112 kB 50.9 MB/s 
     |████████████████████████████████| 149 kB 51.0 MB/s 


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [ ]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [ ]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_myPers_OPN"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-26 08:08:26,767] A new study created in RDB with name: SIMCSE_BERT_myPers_OPN


A new study created in RDB with name: SIMCSE_BERT_myPers_OPN


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weigh

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.562775,0.747984,0.453841,0.762744,0.512369
2,0.567700,0.550678,0.765121,0.580592,0.707627,0.579684
3,0.452400,0.733500,0.741935,0.641533,0.652987,0.634610
4,0.231800,1.291667,0.746472,0.623134,0.652731,0.613298
4,0.095900,1.350280,0.748992,0.629726,0.657765,0.619483


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       510
           1       0.74      1.00      0.85      1474

    accuracy                           0.74      1984
   macro avg       0.37      0.50      0.43      1984
weighted avg       0.55      0.74      0.63      1984

[[   0  510]
 [   0 1474]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Trial 49 pruned. 


[I 2021-12-27 04:58:49,788] A new study created in memory with name: no-name-a0ce318f-fd0f-4bfa-acab-7495344456e6


A new study created in memory with name: no-name-a0ce318f-fd0f-4bfa-acab-7495344456e6


In [ ]:
storage_name

'sqlite:///SIMCSE_BERT_myPers_OPN.db'

In [ ]:
study_name

'SIMCSE_BERT_myPers_OPN'

In [ ]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-27 04:58:49,903] Using an existing study with name 'SIMCSE_BERT_myPers_OPN' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_myPers_OPN' instead of creating a new one.


In [ ]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.629726,7.938171e-05,4.417547,16,1.011780e-07,COMPLETE
1,1,0.426258,9.105113e-05,4.940370,8,1.397096e-05,COMPLETE
2,2,0.426258,2.492401e-01,4.589276,2,3.513224e-02,COMPLETE
3,3,0.426258,2.094824e-03,1.110429,8,2.095970e-02,COMPLETE
4,4,0.426258,1.474898e-04,1.096962,2,5.701092e-09,COMPLETE
5,5,0.426258,5.193582e-03,2.657387,8,1.268058e-01,COMPLETE
6,6,0.426258,2.571224e-07,2.224512,8,7.451598e-08,COMPLETE
7,7,0.426258,1.406138e-01,1.210753,4,3.498292e-06,COMPLETE
8,8,0.426258,1.705713e-02,3.567578,16,3.848108e-06,COMPLETE
9,9,0.426258,1.774046e-03,1.529049,2,8.088638e-10,COMPLETE


In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
best_run

BestRun(run_id='0', objective=0.6297259011085378, hyperparameters={'learning_rate': 7.938170684431916e-05, 'num_train_epochs': 4.417547496496372, 'per_device_train_batch_size': 16, 'weight_decay': 1.0117798083724145e-07})

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_edf(study)